In [1]:
import numpy as np
import matplotlib.pyplot as plt
import scipy as sp
from sklearn.model_selection import train_test_split
from sklearn import datasets
from sklearn import svm

import pandas as pd
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestClassifier  # Example model
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report


In [17]:
df = pd.read_csv('gaia_matched/JOHNSTON_MDwarfs_Crossmatch.csv')  # Replace 'your_file.csv' with your filename
pd.set_option('display.max_columns', None)  # Show all columns
# print(df.head())  # Display the first few rows to check the data
print(len(df))

#drop columns which are entirely strings
#strx = df.drop(columns = (['Designation', '2MASS Number', 'STAT', 'PM?', 'M?', 'source_id','DISC PUBL','duplicated_source']))

columns_to_clean = ['phot_g_mean_flux', 'phot_bp_mean_flux','phot_rp_mean_flux', 'phot_bp_rp_excess_factor', 
                   'bp_rp','phot_g_mean_flux_error', 'phot_bp_mean_flux_error', 'phot_g_mean_mag', 'phot_bp_mean_mag', 'phot_rp_mean_mag', 
                   'phot_g_mean_mag_error', 'phot_bp_mean_mag_error', 'phot_rp_mean_mag_error',
                   'phot_g_mean_mag_corrected', 'phot_g_mean_mag_error_corrected', 'phot_g_mean_flux_corrected', 'phot_bp_rp_excess_factor_corrected']

#tried to organize these a little bit for future use if needed
flux_col = ['phot_g_mean_flux', 'phot_bp_mean_flux','phot_rp_mean_flux']
#phot_bp_rp_excess_factor,bp_rp]  # List the columns to process
err_col = ['phot_g_mean_flux_error', 'phot_bp_mean_flux_error', '']
mag_col = ['phot_g_mean_mag', 'phot_bp_mean_mag', 'phot_rp_mean_mag']
mag_err_col = ['phot_g_mean_mag_error', 'phot_bp_mean_mag_error', 'phot_rp_mean_mag_error']
#mag_corr_col = ['phot_g_mean_mag_corrected', phot_g_mean_mag_error_corrected,phot_g_mean_flux_corrected,phot_bp_rp_excess_factor_corrected
#strx = df.drop(columns = (['parallax','parallax_error','parallax_over_error','pm','pmra','pmra_error',
                             #'pmdec','pmdec_error','ruwe']))

df[columns_to_clean] = df[columns_to_clean].apply(pd.to_numeric, errors='coerce')

strx = df[columns_to_clean]

#Count NaNs in each column
nan_count = strx.isnull().sum()

# Display columns with NaNs
print(nan_count[nan_count > 0])
print(np.max(nan_count) / len(strx))

# drop the exceptional columns that still have NaNs
NaNx_clean = strx.dropna()
m_indices = NaNx_clean.index.tolist()
print(len(NaNx_clean))

# # Select columns with string (object) data
# string_columns = strx.select_dtypes(include=['object'])
# print(string_columns)

# string_positions = strx.applymap(lambda x: isinstance(x, str)).stack()
# print(strx.stack()[string_positions])

# # print(strx['W3'][2].dtype)

597
phot_g_mean_flux                       1
phot_bp_mean_flux                     24
phot_rp_mean_flux                     20
phot_bp_rp_excess_factor              24
bp_rp                                 24
phot_g_mean_flux_error                 1
phot_bp_mean_flux_error               24
phot_g_mean_mag                        1
phot_bp_mean_mag                      24
phot_rp_mean_mag                      20
phot_g_mean_mag_error                  1
phot_bp_mean_mag_error                24
phot_rp_mean_mag_error                20
phot_g_mean_mag_corrected              1
phot_g_mean_mag_error_corrected        1
phot_g_mean_flux_corrected             1
phot_bp_rp_excess_factor_corrected    24
dtype: int64
0.04020100502512563
573


In [18]:
X = NaNx_clean  # Features
y = df['SPEC'][m_indices]  # Labels

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [20]:
df = pd.read_csv('gaia_matched/JOHNSTON_NotMDwarfs_Crossmatch.csv')  # Replace 'your_file.csv' with your filename
pd.set_option('display.max_columns', None)
print(len(df))

columns_to_clean = ['phot_g_mean_flux', 'phot_bp_mean_flux','phot_rp_mean_flux', 'phot_bp_rp_excess_factor', 
                   'bp_rp','phot_g_mean_flux_error', 'phot_bp_mean_flux_error', 'phot_g_mean_mag', 'phot_bp_mean_mag', 'phot_rp_mean_mag', 
                   'phot_g_mean_mag_error', 'phot_bp_mean_mag_error', 'phot_rp_mean_mag_error',
                   'phot_g_mean_mag_corrected', 'phot_g_mean_mag_error_corrected', 'phot_g_mean_flux_corrected', 'phot_bp_rp_excess_factor_corrected']

df[columns_to_clean] = df[columns_to_clean].apply(pd.to_numeric, errors='coerce')

strx = df[columns_to_clean]

nan_count = strx.isnull().sum()
print(nan_count[nan_count > 0])
print(np.max(nan_count) / len(strx))

NaNx_clean = strx.dropna()
notm_indices = NaNx_clean.index.tolist()
print(len(NaNx_clean))

1100
phot_g_mean_flux                      16
phot_bp_mean_flux                     55
phot_rp_mean_flux                     27
phot_bp_rp_excess_factor              55
bp_rp                                 55
phot_g_mean_flux_error                16
phot_bp_mean_flux_error               55
phot_g_mean_mag                       16
phot_bp_mean_mag                      55
phot_rp_mean_mag                      27
phot_g_mean_mag_error                 16
phot_bp_mean_mag_error                55
phot_rp_mean_mag_error                27
phot_g_mean_mag_corrected             16
phot_g_mean_mag_error_corrected       16
phot_g_mean_flux_corrected            16
phot_bp_rp_excess_factor_corrected    55
dtype: int64
0.05
1045


In [21]:
X = NaNx_clean  # Features
y = df['SPEC'][notm_indices]  # Labels

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [25]:
df = pd.read_csv('gaia_matched/RANDOM_Stars_Crossmatch.csv')  # Replace 'your_file.csv' with your filename
pd.set_option('display.max_columns', None)
print(len(df))

columns_to_clean = ['phot_g_mean_flux', 'phot_bp_mean_flux','phot_rp_mean_flux', 'phot_bp_rp_excess_factor', 
                   'bp_rp','phot_g_mean_flux_error', 'phot_bp_mean_flux_error', 'phot_g_mean_mag', 'phot_bp_mean_mag', 'phot_rp_mean_mag', 
                   'phot_g_mean_mag_error', 'phot_bp_mean_mag_error', 'phot_rp_mean_mag_error',
                   'phot_g_mean_mag_corrected', 'phot_g_mean_mag_error_corrected', 'phot_g_mean_flux_corrected', 'phot_bp_rp_excess_factor_corrected']

df['SPEC'] = 'ND'

df[columns_to_clean] = df[columns_to_clean].apply(pd.to_numeric, errors='coerce')

strx = df[columns_to_clean]

nan_count = strx.isnull().sum()
print(nan_count[nan_count > 0])
print(np.max(nan_count) / len(strx))

NaNx_clean = strx.dropna()
rand_indices = NaNx_clean.index.tolist()
print(len(NaNx_clean))

70902
phot_g_mean_flux                       168
phot_bp_mean_flux                     5569
phot_rp_mean_flux                     5149
phot_bp_rp_excess_factor              5611
bp_rp                                 5611
phot_g_mean_flux_error                 168
phot_bp_mean_flux_error               5569
phot_g_mean_mag                        168
phot_bp_mean_mag                      5569
phot_rp_mean_mag                      5149
phot_g_mean_mag_error                  168
phot_bp_mean_mag_error                5569
phot_rp_mean_mag_error                5149
phot_g_mean_mag_corrected              168
phot_g_mean_mag_error_corrected        168
phot_g_mean_flux_corrected             168
phot_bp_rp_excess_factor_corrected    5611
dtype: int64
0.07913740091957913
65291


In [26]:
X = NaNx_clean  # Features
y = df['SPEC'][rand_indices]  # Labels

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [27]:
# Machine learning for the cleaned spectral data
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# Train-test split
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.2, random_state=42)

# KNN classifier
knn = KNeighborsClassifier(n_neighbors=5, metric='euclidean')
knn.fit(X_train, y_train)

# Predictions
y_pred = knn.predict(X_test)